# 02. ATP Match Centre 2D Court Vision Scraper

Notebook will contain codes and functions for scraping almost all service data from a ATP court vision page (where available) for a specific match.
Also a rough pipeline to enable scraping at a much larger scale e.g. all matches per valid tournament

## 1. Imports and Setup

In [2]:
# Standard math libraries
import numpy as np
import scipy as sp
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

# Web-scraping utitilies
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
# headers = {'User-Agent': 
#            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'} 
import re
import json

import sys
from time import sleep


# Selenium Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.microsoft  import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [3]:
from selenium.webdriver.edge.options import Options

# Setting selenium options
options = Options()
options.headless = False

In [3]:
service = EdgeService(executable_path=EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service, options=options)

In [4]:
# ATP match centre URL
url = "https://www.atptour.com/en/scores/stats-centre/archive/2022/328/ms002?tab=CourtVision"

In [6]:
# Get URL
driver.get(url)
# Maxmimise the browser window
driver.maximize_window()

In [7]:
# Click the 2D-view button
driver.find_elements(By.XPATH, "//button[@class='button-wrapper-MC toggle-btn-wrapper first-btn ']")[0].click()

In [8]:
# Click on the shot type dropdown then select the required serve stat or other
driver.find_elements(By.XPATH, "//div[@class='select-arrow']")[-1].click()
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(),'First serve')]"))).click()

In [10]:
# The div element containing the clickable player names to toggle stats between them
players_div = driver.find_elements(By.XPATH, "//div[@class='playersDiv']")[0]
# Click on Charlie's Name
player_buttons = players_div.find_elements(By.XPATH, ".//span[@class='playerName']")
player_buttons[0].click()
#player_buttons[1].click()

In [9]:
# Get the player names
player_1 = driver.find_element(By.XPATH, "//div[@class='player-left-name']").text.replace('\n', ' ')
player_2 = driver.find_element(By.XPATH, "//div[@class='player-right-name']").text.replace('\n', ' ')
print(player_1, player_2)

Novak Djokovic Sebastian Korda


In [11]:
# Get all the clickable point circle elements
point_circs = driver.find_elements(By.CSS_SELECTOR, "svg[id^='point']")#point-1_1_4_1

In [4]:
def select_shot_type(shot_type):
    """
    Toggle between the different shot selection types on the Match Centre webapp
    
    """
    if shot_type not in ["First serve", "Second serve"]:
        sys.exit('Please provide a valid shot type selection')
    # Click on the shot type dropdown then select the required serve stat or other
    driver.find_elements(By.XPATH, "//div[@class='select-arrow']")[-1].click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(),'{shot_type}')]"))).click()

In [13]:
# Service Box Dims

servebox_html = driver.find_elements(By.CSS_SELECTOR, "#serviceBox")[0].get_attribute('outerHTML')
box_x, box_y = float(servebox_html.split(' width="')[1].split('" height="')[0]), float(servebox_html.split(' height="')[1].split('" stroke="')[0])

### Function to loop every point and gather data

In [91]:
def scrape_serve_data(driver, player1, player2, player_serving, shot_type, swap_players=False):
    """
    Scrape serve data from a single serve type from Court Vision 2D
    
    """
    driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view 

    # Get all the "point" elements i.e. the dot-separators and the clickable point circle elements
    all_points = driver.find_elements(By.CSS_SELECTOR, "svg[id^='point'], #dot-separator")

    # Get all the clickable point circle elements
    point_circs = driver.find_elements(By.CSS_SELECTOR, "svg[id^='point']")#point-1_1_4_1

    print("found points")

    # Initialise all data lists
    x_list = [] # x-coords
    y_list = [] # y-coords
    outcomes_list = [] # point outcome

    p1_score_G_list = []
    p2_score_G_list = []
    p1_setswon_list = [] # no. of sets won so far in match
    p2_setswon_list = []
    p1_setscore_list = [] # no. of games won so far in set
    p2_setscore_list = [] 
    set_N_list = [] # contains the set of associated point
    point_list = [] # contains point numbers


    speed_list = [] # speeds list
    type_list = [] # type of serve
    ral_len_list = [] # len of rally
    height_grd_list = [] # "height above ground"
    height_bounce_list = [] # "bounce height"
    outcome_type_list = []

    # Loop through every point on the timeline, click and grab info
    for point in point_circs[:]:
        
        # Being nice and randomly sleeping
        sleeptime = np.random.uniform(0, 1)
        sleep(sleeptime)

        # Get the current point number 
        point_n = all_points.index(point) + 1
        point_list.append(point_n)

        # This scrolls back to where the point-bar is viewable (using the actual element itself gets it blocked by the navbar sadly)
        driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
        # Clicking on the point circle highlights a point/scatter on the point, allowing it to be clicked with
        # precedence over other overlapping points
        point.click()

        # Get the selected highlighted ball element
        #selected_ball = driver.find_element(By.CSS_SELECTOR, "g[id='plottedBallsSelected']")
        try:
            selected_ball = driver.find_elements(By.CSS_SELECTOR, "g[id='plottedBallsSelected'] > g[class^='court-ball-']")[-1]
            # Click to bring up the mini stats window
            selected_ball.click()

            # Get ball's "x" (depth) coord
            x = float(selected_ball.get_attribute('outerHTML').split('<use x="')[1].split('" y="')[0])
            # Get ball's "y" coord
            y = float(selected_ball.get_attribute('outerHTML').split('y="')[1].split("\"")[0])
            # Get the outcome of this point (Won/Lost/Ace)
            outcome = selected_ball.get_attribute('outerHTML').split('href="#')[-1].split("\"")[0].split('Selected')[-1]


            # Find the needed elements on the stats widget and assign to variables
            # Point context/metadata
            # Current game score
            p1_score_G, p2_score_G = driver.find_element(By.CSS_SELECTOR, "div[class^='game-scores']").text.split('\n')

            # Get number of sets won per player and current set score
            p1_setswon = 0
            p2_setswon = 0

            # Set score elements
            sets_eles = driver.find_elements(By.CSS_SELECTOR, "div[class^='set-scores-']")

            # Current set number
            set_N = len(sets_eles)

            # Get the played sets' scores and simplify to just sets won
            if len(sets_eles) > 1:
                for set in sets_eles[:-1]:
                    p1_setscore, p2_setscore = [ int(x.text) for x in set.find_elements(By.TAG_NAME, "span[class^='set-value']") ]
                    if p1_setscore > p2_setscore:
                        p1_setswon += 1
                    else:
                        p2_setswon += 1

            # Current current set's score
            p1_setscore, p2_setscore = [ int(x.text) for x in sets_eles[-1].find_elements(By.TAG_NAME, "span[class^='set-value']") ]


            # Now get the actual shot info (speed, subsequent rally length)
            # Speed in KMH
            speed = int(driver.find_elements(By.XPATH, ".//div[@class='speed-data']")[1].text.split('\n')[0])

            # Serve Type 
            serve_type = driver.find_elements(By.XPATH, ".//div[@class='md-block md-serve-type']")[0].text.split('\n')[-1]

            # Rally Length
            if outcome == "Ace":
                ral_len = 1
                try:
                    height_grd = float(driver.find_element(By.CSS_SELECTOR, "div.md-block.md-height-above-ground > div.value").text.split(' ')[0])
                except:
                    height_grd = driver.find_element(By.CSS_SELECTOR, "div.md-block.md-height-above-ground > div.value").text.split(' ')[0]
                try:
                    height_bounce = float(driver.find_element(By.CSS_SELECTOR, "div.md-block.md-bounce-height > div.value").text.split(' ')[0])
                except ValueError:
                    height_bounce = driver.find_element(By.CSS_SELECTOR, "div.md-block.md-bounce-height > div.value").text.split(' ')[0]

            else:
                ral_len = int(driver.find_elements(By.XPATH, ".//div[@class='md-block md-rally-length']")[0].text.split('\n')[-1].split(' Shot')[0])
                height_grd = '-'
                height_bounce = '-'

            # Rally Outcome Type
            try:
                outcome_type = driver.find_elements(By.XPATH, ".//div[@class='shot-description']")[0].text.split("'S ")[1].title()
            except:
                outcome_type = "-"

        #Some points may have missing court vision data
        except: 
            outcome = driver.find_element(By.CSS_SELECTOR, "svg[class^='selected-ball'] > text:nth-child(3) ").text
            if outcome == "W":
                outcome = "Winner"
            else:
                outcome = "Lost"
            p1_score_G,p2_score_G = driver.find_element(By.CSS_SELECTOR, "svg[class^='selected-ball'] > text:nth-child(4) ").text.replace('(','').replace(')','').split(' - ')
            # Other data no choide but to be '-'
            x = y = np.nan
            p1_setswon = p2_setswon = p1_setscore = p2_setscore = set_N = '-'
            speed = serve_type = ral_len = height_bounce = height_bounce = outcome_type = "-"
        
        # Append data to the lists
        x_list.append(x)
        y_list.append(y)
        outcomes_list.append(outcome)

        p1_score_G_list.append(p1_score_G)
        p2_score_G_list.append(p2_score_G)
        p1_setswon_list.append(p1_setswon)
        p2_setswon_list.append(p2_setswon)
        p1_setscore_list.append(p1_setscore)
        p2_setscore_list.append(p2_setscore)
        set_N_list.append(set_N)

        speed_list.append(speed)
        type_list.append(serve_type)
        ral_len_list.append(ral_len)
        height_grd_list.append(height_grd)
        height_bounce_list.append(height_bounce)
        outcome_type_list.append(outcome_type)

    shot_type_list = [shot_type]*len(x_list)
    player_serving_list = [player_serving]*len(x_list)

    # Transform the x,y data to real court dims
    # Transformed XY Data
    if np.nanmean(x_list) < 0:
        x_trans = (np.array(y_list)*1/box_y)*8.23/2
        y_trans = (np.array(x_list)*-1/box_x)*6.4
    else:
        x_trans = (np.array(y_list)*-1/box_y)*8.23/2
        y_trans = (np.array(x_list)*1/box_x)*6.4

    if swap_players == False:
        df = pd.DataFrame({
            "Player1": [player1]*len(x_list), "Player2":[player2]*len(x_list), "Set": set_N_list, "Point": point_list, "Player1_Sets":p1_setswon_list, "Player2_Sets":p2_setswon_list, \
            "Player1_Game": p1_setscore_list, "Player2_Game": p2_setscore_list, "Player1_Score": p1_score_G_list , "Player2_Score": p2_score_G_list, \
            "Serving":player_serving_list, "Shot_Type": shot_type_list, \
            "X":x_trans, "Y": y_trans, "Speed_kmh": speed_list, "Serve_Type": type_list, "Rally_Length":ral_len_list, "Height_Above_Ground":height_grd_list, "Bounce_Height": height_bounce_list, \
            "Outcome": outcomes_list, "Outcome_Type":outcome_type_list,})

    else: # i.e. the player 1 and 2 name orders contradict between the stats page and court vision
        df = pd.DataFrame({
            "Player1": [player2]*len(x_list), "Player2":[player1]*len(x_list), "Set": set_N_list, "Point": point_list, "Player1_Sets":p2_setswon_list, "Player2_Sets":p1_setswon_list, \
            "Player1_Game": p2_setscore_list, "Player2_Game": p1_setscore_list, "Player1_Score": p2_score_G_list , "Player2_Score": p1_score_G_list, \
            "Serving":player_serving_list, "Shot_Type": shot_type_list, \
            "X":x_trans, "Y": y_trans, "Speed_kmh": speed_list, "Serve_Type": type_list, "Rally_Length":ral_len_list, "Height_Above_Ground":height_grd_list, "Bounce_Height": height_bounce_list, \
            "Outcome": outcomes_list, "Outcome_Type":outcome_type_list,})

    return df



In [82]:
driver.find_element(By.CSS_SELECTOR, "svg[class^='selected-ball'] > text:nth-child(3) ").text
driver.find_element(By.CSS_SELECTOR, "svg[class^='selected-ball'] > text:nth-child(4) ").text.replace('(','').replace(')','').split(' - ')

['G', '40']

In [6]:
## Function to scrape all service data (both players) for a given match (url)

def scrape_match_serves(url, tournament, year, round, player1, player2):
    """
    Full function for scraping all service points in a given match
    """

    # Resets Driver
    service = EdgeService(executable_path=EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=options)

    # Get URL
    driver.get(url)

    # Maxmimise the browser window
    driver.maximize_window()

    # Click the 2D-view button
    driver.find_elements(By.XPATH, "//button[@class='button-wrapper-MC toggle-btn-wrapper first-btn ']")[0].click()

    # Get Service Box Dims
    servebox_html = driver.find_elements(By.CSS_SELECTOR, "#serviceBox")[0].get_attribute('outerHTML')
    box_x, box_y = float(servebox_html.split(' width="')[1].split('" height="')[0]), float(servebox_html.split(' height="')[1].split('" stroke="')[0])

    # Get Player Index Info first, then loop through players, then through serve type to scrape data
    # The div element containing the clickable player names to toggle stats between them
    players_div = driver.find_elements(By.XPATH, "//div[@class='playersDiv']")[0]
    # Find the corresponding buttons for the players...the winner isn't necessarily the player1 in the Court Vision GUI
    # Start from player1 first, i.e. click the correct button order
    player_buttons = players_div.find_elements(By.XPATH, ".//span[@class='playerName']")

    print("Done player buttons")

    # Get player1's last name in CAPs
    player1_lastname = player1.split(" ")[-1].upper()
    # Get the last name of player button 1, in CAPs
    button1_lastname = player_buttons[0].text.split('. ')[-1]
    # If the winner/loser player order matches up with the 2D court-vision's left-right player buttons, set player list to be iterated to player1,2 
    if player1_lastname == button1_lastname:
        players = [player1, player2]
        player_i = 1 # Player Index
        swap_players = False
    else:
        players = [player2, player1] # swap the player list if the names do not match
        player_i = 2
        swap_players = True

    df_serves_list = []

    for button in player_buttons:
        # Click on selected player button
        button.click()

        for shot_type in ["First serve", "Second serve"]:
            # Select the serve type
            select_shot_type(shot_type)
            # Reset view..again
            driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
            # Scrape service data for the current player and type
            df_serves = scrape_serve_data(driver, players[0], players[1], player_i, shot_type, swap_players)
            df_serves_list.append(df_serves)

            # Reset view
            driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
            # Scroll back to the start of the points separator time chart by changing the Tabs to reset the timeline view
            # Scrolls back to where the Stats/Vision Tabs are visible
            driver.find_element(By.CSS_SELECTOR, "#match-stats-container > div.modal-scores-header > a").location_once_scrolled_into_view
            driver.find_element(By.CSS_SELECTOR, "#tabStats").click()
            driver.find_element(By.CSS_SELECTOR, "#tabCourtVision").click()
            # Reset view
            driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
            
        # Swap player index for next iterations
        if player_i == 1:
            player_i = 2
        else:
            player_i = 1


    # Combine all 4 dataframes into one
    df_serves_all = pd.concat(df_serves_list).sort_values('Point')
    df_serves_all = df_serves_all.reset_index(drop=True)

            
    # Add additional metadata columns
    # Add a column for Year
    df_serves_all.insert(0, "Year", [year]*len(df_serves_all))
    # Add a column for tournament name
    df_serves_all.insert(1, "Tournament", [tournament]*len(df_serves_all))
    # Add a column for tournament round
    df_serves_all.insert(2, "Round", [round]*len(df_serves_all))

    # Quit Browser
    driver.quit()

    return df_serves_all

In [93]:
driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
#select_shot_type("First serve")
select_shot_type("Second serve")

In [87]:
df_serve = scrape_serve_data(player_2, player_1, 2, "1st serve")

In [94]:
df_2nd = scrape_serve_data(player_2, player_1, 2, shot_type="2nd Serve")

In [96]:
df_all = pd.concat([df_serve,df_2nd]).sort_values('Point')

In [97]:
df_all.head()

,Player1,Player2,Set,Point,Player1_Sets,Player2_Sets,Player1_Game,Player2_Game,Player1_Score,Player2_Score,...,Shot_Type,X,Y,Speed_kmh,Serve_Type,Rally_Length,Height_Above_Ground,Bounce_Height,Outcome,Outcome_Type
0,Felix Auger-Aliassime,Carlos Alcaraz,1,7,0,0,0,1,15,0,...,1st serve,-0.219266,5.890,203,Slice,1,-,-,Winner,Forced Error
0,Felix Auger-Aliassime,Carlos Alcaraz,1,8,0,0,0,1,15,15,...,2nd Serve,2.608169,4.280,151,Pronated,4,-,-,Lost,Winner
1,Felix Auger-Aliassime,Carlos Alcaraz,1,9,0,0,0,1,30,15,...,1st serve,-0.487592,5.857,194,Slice,9,-,-,Winner,Forced Error
2,Felix Auger-Aliassime,Carlos Alcaraz,1,10,0,0,0,1,30,30,...,1st serve,0.474577,5.936,186,Slice,4,-,-,Lost,Unforced Error
3,Felix Auger-Aliassime,Carlos Alcaraz,1,11,0,0,0,1,40,30,...,1st serve,-1.965388,5.324,198,Slice,9,-,-,Winner,Forced Error


In [27]:
driver.quit()

In [10]:
url= "https://www.atptour.com/en/scores/stats-centre/archive/2023/2843/ms001?tab=CourtVision"
#"https://www.atptour.com/en/scores/stats-centre/archive/2023/2843/ms001?tab=CourtVision"

In [25]:
df_serves_match = scrape_match_serves(url, "Adelaide International 1", 2023, "Final", "Novak Djokovic", "Sebastian Korda")

IndexError: list index out of range

In [101]:
driver.quit()

In [7]:
player1 = "Novak Djokovic"
player2 = "Sebastian Korda"

In [97]:
# Resets Driver
service = EdgeService(executable_path=EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service, options=options)

# Get URL
driver.get(url)

# Maxmimise the browser window
driver.maximize_window()

#driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
# Click the 2D-view button
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='button-wrapper-MC toggle-btn-wrapper first-btn ']"))).click()
#driver.find_elements(By.XPATH, "//button[@class='button-wrapper-MC toggle-btn-wrapper first-btn ']")[0].click()

# Get Service Box Dims
servebox_html = driver.find_elements(By.CSS_SELECTOR, "#serviceBox")[0].get_attribute('outerHTML')
box_x, box_y = float(servebox_html.split(' width="')[1].split('" height="')[0]), float(servebox_html.split(' height="')[1].split('" stroke="')[0])

# Get Player Index Info first, then loop through players, then through serve type to scrape data
# The div element containing the clickable player names to toggle stats between them
players_div = driver.find_elements(By.XPATH, "//div[@class='playersDiv']")[0]
# Find the corresponding buttons for the players...the winner isn't necessarily the player1 in the Court Vision GUI
# Start from player1 first, i.e. click the correct button order
player_buttons = players_div.find_elements(By.XPATH, ".//span[@class='playerName']")

print("Done player buttons")

# Get player1's last name in CAPs
player1_lastname = player1.split(" ")[-1].upper()
# Get the last name of player button 1, in CAPs
button1_lastname = player_buttons[0].text.split('. ')[-1]
# If the winner/loser player order matches up with the 2D court-vision's left-right player buttons, set player list to be iterated to player1,2 
if player1_lastname == button1_lastname:
    players = [player1, player2]
    player_i = 1 # Player Index
    swap_players = False
else:
    players = [player2, player1] # swap the player list if the names do not match
    player_i = 2
    swap_players = True

df_serves_list = []

Done player buttons


In [99]:
df_serves = scrape_serve_data(driver, players[0], players[1], 2, "Second Serve", False)

found points


In [95]:
# Resets Driver
service = EdgeService(executable_path=EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service, options=options)

# Get URL
driver.get(url)

# Maxmimise the browser window
driver.maximize_window()

#driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
# Click the 2D-view button
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='button-wrapper-MC toggle-btn-wrapper first-btn ']"))).click()
#driver.find_elements(By.XPATH, "//button[@class='button-wrapper-MC toggle-btn-wrapper first-btn ']")[0].click()

# Get Service Box Dims
servebox_html = driver.find_elements(By.CSS_SELECTOR, "#serviceBox")[0].get_attribute('outerHTML')
box_x, box_y = float(servebox_html.split(' width="')[1].split('" height="')[0]), float(servebox_html.split(' height="')[1].split('" stroke="')[0])

# Get Player Index Info first, then loop through players, then through serve type to scrape data
# The div element containing the clickable player names to toggle stats between them
players_div = driver.find_elements(By.XPATH, "//div[@class='playersDiv']")[0]
# Find the corresponding buttons for the players...the winner isn't necessarily the player1 in the Court Vision GUI
# Start from player1 first, i.e. click the correct button order
player_buttons = players_div.find_elements(By.XPATH, ".//span[@class='playerName']")

print("Done player buttons")

# Get player1's last name in CAPs
player1_lastname = player1.split(" ")[-1].upper()
# Get the last name of player button 1, in CAPs
button1_lastname = player_buttons[0].text.split('. ')[-1]
# If the winner/loser player order matches up with the 2D court-vision's left-right player buttons, set player list to be iterated to player1,2 
if player1_lastname == button1_lastname:
    players = [player1, player2]
    player_i = 1 # Player Index
    swap_players = False
else:
    players = [player2, player1] # swap the player list if the names do not match
    player_i = 2
    swap_players = True

df_serves_list = []

max_attempts = 3
attempt = 1
for i in range(2):
    for shot_type in ["First serve", "Second serve"]:
        players_div = driver.find_elements(By.XPATH, "//div[@class='playersDiv']")[0]
        player_buttons = players_div.find_elements(By.XPATH, ".//span[@class='playerName']")
        player_buttons[i].click()
        # Select the serve type
        select_shot_type(shot_type)
        # Reset view..again
        driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
        # Scrape service data for the current player and type
        df_serves = scrape_serve_data(driver, players[0], players[1], player_i, shot_type, swap_players)
        df_serves_list.append(df_serves)

        # Reset view
        driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
        # Scroll back to the start of the points separator time chart by changing the Tabs to reset the timeline view
        # Scrolls back to where the Stats/Vision Tabs are visible
        driver.find_element(By.CSS_SELECTOR, "#match-stats-container > div.modal-scores-header > a").location_once_scrolled_into_view
        driver.find_element(By.CSS_SELECTOR, "#tabStats").click()
        driver.find_element(By.CSS_SELECTOR, "#tabCourtVision").click()
        # Reset view
        driver.find_element(By.CSS_SELECTOR, "div[class='atp-stats-logo']").location_once_scrolled_into_view
        

    # Swap player index for next iterations
    if player_i == 1:
        player_i = 2
    else:
        player_i = 1
    

Done player buttons
found points
found points
found points
found points


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"svg[class^='selected-ball'] > text:nth-child(3) "}
  (Session info: MicrosoftEdge=109.0.1518.52)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF6C9FD0A12+15186]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6C9F69562+828946]
	(No symbol) [0x00007FF6C9C2ED90]
	(No symbol) [0x00007FF6C9C72225]
	(No symbol) [0x00007FF6C9C723AC]
	(No symbol) [0x00007FF6C9CAE087]
	(No symbol) [0x00007FF6C9C91F8F]
	(No symbol) [0x00007FF6C9C64C3E]
	(No symbol) [0x00007FF6C9CAB513]
	(No symbol) [0x00007FF6C9C91D23]
	(No symbol) [0x00007FF6C9C63B80]
	(No symbol) [0x00007FF6C9C62B0E]
	(No symbol) [0x00007FF6C9C64344]
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF6C9E4B190+182768]
	(No symbol) [0x00007FF6C9D1EF45]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6C9EA94CA+4236

In [92]:
x = y = np.nan

In [42]:
pd.concat(df_serves_list).sort_values("Point")

,Player1,Player2,Set,Point,Player1_Sets,Player2_Sets,Player1_Game,Player2_Game,Player1_Score,Player2_Score,...,Shot_Type,X,Y,Speed_kmh,Serve_Type,Rally_Length,Height_Above_Ground,Bounce_Height,Outcome,Outcome_Type
0,Novak Djokovic,Sebastian Korda,1,1,0,0,0,0,15,0,...,First serve,-0.498606,5.866,205,Flat,15,-,-,Winner,Winner
0,Novak Djokovic,Sebastian Korda,1,2,0,0,0,0,15,15,...,Second serve,0.623758,4.896,172,Slice,2,-,-,Lost,Winner
1,Novak Djokovic,Sebastian Korda,1,3,0,0,0,0,30,15,...,Second serve,-3.489240,5.679,168,Slice,1,-,-,Winner,Forced Error
2,Novak Djokovic,Sebastian Korda,1,4,0,0,0,0,40,15,...,Second serve,1.893301,3.765,139,Kick,3,-,-,Winner,Forced Error
3,Novak Djokovic,Sebastian Korda,1,5,0,0,1,0,G,15,...,Second serve,-1.294573,4.506,149,Kick,3,-,-,Winner,Forced Error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Novak Djokovic,Sebastian Korda,3,221,1,1,5,4,0,15,...,First serve,-0.237288,5.421,201,Flat,1,-,-,Winner,Forced Error
72,Novak Djokovic,Sebastian Korda,3,223,1,1,5,4,15,30,...,First serve,-0.484589,4.867,202,Flat,2,-,-,Lost,Unforced Error
73,Novak Djokovic,Sebastian Korda,3,224,1,1,5,4,15,40,...,First serve,3.493245,5.867,203,Flat,7,-,-,Winner,Forced Error
74,Novak Djokovic,Sebastian Korda,3,225,1,1,5,4,30,40,...,First serve,-0.575700,5.200,203,Flat,4,-,-,Lost,Forced Error


In [29]:
# Scrape service data for the current player and type
df_serves = scrape_serve_data(player1, player2, player_i, shot_type, swap_players)

In [68]:
df_serves

,Player1,Player2,Set,Point,Player1_Sets,Player2_Sets,Player1_Game,Player2_Game,Player1_Score,Player2_Score,...,Shot_Type,X,Y,Speed_kmh,Serve_Type,Rally_Length,Height_Above_Ground,Bounce_Height,Outcome,Outcome_Type
0,Sebastian Korda,Novak Djokovic,1,2,0,0,0,0,15,15,...,Second serve,0.623758,4.896,172,Slice,2,-,-,Lost,Winner
1,Sebastian Korda,Novak Djokovic,1,3,0,0,0,0,30,15,...,Second serve,-3.489240,5.679,168,Slice,1,-,-,Winner,Forced Error
2,Sebastian Korda,Novak Djokovic,1,4,0,0,0,0,40,15,...,Second serve,1.893301,3.765,139,Kick,3,-,-,Winner,Forced Error
3,Sebastian Korda,Novak Djokovic,1,5,0,0,1,0,G,15,...,Second serve,-1.294573,4.506,149,Kick,3,-,-,Winner,Forced Error
4,Sebastian Korda,Novak Djokovic,1,25,0,0,2,2,15,0,...,Second serve,-0.648788,4.173,151,Kick,3,-,-,Winner,Forced Error
5,Sebastian Korda,Novak Djokovic,1,38,0,0,3,3,30,0,...,Second serve,2.550098,5.012,144,Kick,5,-,-,Winner,Unforced Error
6,Sebastian Korda,Novak Djokovic,1,40,0,0,4,3,G,0,...,Second serve,0.261318,4.941,173,Slice,1,-,-,Winner,Forced Error
7,Sebastian Korda,Novak Djokovic,1,46,0,0,4,4,15,15,...,Second serve,3.364087,3.913,138,Pronated,2,-,-,Lost,Winner
8,Sebastian Korda,Novak Djokovic,1,49,0,0,4,5,15,G,...,Second serve,-1.085319,3.937,146,Kick,12,-,-,Lost,Winner
9,Sebastian Korda,Novak Djokovic,1,60,0,0,5,5,40,0,...,Second serve,-3.395125,4.879,167,Slice,1,-,-,Winner,Forced Error
